I've been reading through http://www.cs.nott.ac.uk/~pszgmh/fold.pdf.  TL;DR; Replace recursive functions with folds on finite collections.  It made me think about the Fibonacci function (everyone's pet recursion function).

In [39]:
from functools import reduce

def fib(n):
    # `range` is simply a generator, and we ignore its value
    return reduce(lambda x, _: (x[1], x[0] + x[1]), range(n - 1), (1, 1))[0]

%timeit fib(100)

# 18.4 µs ± 16.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

17.9 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
def fibr(n):
    if n in [1, 2]:
        return 1
    return fibr(n -1) + fibr(n - 2)

%timeit fibr(10)

# 15.2 µs ± 5.68 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

15.2 µs ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
def fibfor(n):
    a, b = 1, 1
    if n in [1, 2]:
        return 1
    for i in range(2, n+1):
        a, b = b, a + b
    return a

%timeit fibfor(100)

# 5.72 µs ± 33.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

5.82 µs ± 84.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [72]:
# dropWhile' p = foldr f v
#   where f x (ys, xs) = (if p x then ys else x:xs,  x:xs)
#         v = ([ ],[ ])

def drop_while(pred, items):
    def _drop_while(tup, x):
        ys, xs = tup
        if pred(x):
            return (ys, [x] + xs)
        return ([x] + xs, [x] + xs)
    return reduce(_drop_while, reversed(items), ([], []))[0]

%timeit drop_while(lambda x: x < 42, range(1000))
# 5.27 ms ± 45.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

5.21 ms ± 3.86 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
def drop_it_hot(pred, items):
    for i, v in enumerate(items):
        if pred(v):
            continue
        return items[i:]
    return items

%timeit drop_it_hot(lambda x: x < 42, range(1000))
# 6.49 µs ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

6.49 µs ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [43]:
class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

    def __iter__(self):
        # Traverse the tree, DFS
        yield self
        for child in [self.left, self.right]:
            if not child:
                continue
            for grand in child:
                yield grand
    
    @property
    def children(self):
        if self.left:
            yield self.left
        if self.right:
            yield self.right

    def __repr__(self):
        return f"Node('{self.val}')"

a = Node('a')
a.left = Node('b')
a.right = Node('c')
a.left.left = Node('d')
a.right.left = Node('f')
a.right.right = Node('g')
a.right.left.left = Node('l')
a.right.left.right = Node('m')

print(list(a))

[Node('a'), Node('b'), Node('d'), Node('c'), Node('f'), Node('l'), Node('m'), Node('g')]
